# Interacting with GPIO from MicroBlaze

In [1]:
from pynq.overlays.base import BaseOverlay
import time
import threading
import os
import socket

base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpio(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

In [3]:
LED_Status = [0, 0, 2, 3] #default status
#          = [off/on, solid/blink, cycle/blue/red/green, brightness_level]
Duty_Cycle = [0.04736614270344993, 0.217637640824031, 0.5310492251033824, 1.0] #25%, 50%, 75%, and 100% brightness.
#time_on = duty_cycle * (1/frequency) && time_off = (1-duty_cycle) * (1/frequency)
frequency = 20 #for the PWM Brightness
PWM_on = []
for i in range(4):
    PWM_on.append(Duty_Cycle[i]/frequency)
PWM_off = []
for i in range(4):
    PWM_off.append((1-Duty_Cycle[i])/frequency)
sleep_time = 1
PWM_run_time = frequency*sleep_time
running = True
threads = []

def Change_LED(new_input, element):
    global LED_Status
    reset_gpio()
    LED_Status[element] = new_input
    print(LED_Status)
    
def LED():
    global LED_Status, PWM_on, PWM_off, PWM_run_time, sleep_time
    if LED_Status[0] == 0: #IF OFF
        time.sleep(sleep_time)
    else: #IF ON
        if LED_Status[3] == 3: #full bright (NO PWM)
            if LED_Status[2] == 0: #If NO PWM & Cycle Color
                for i in range(1,4): #cycle write from 1 (blue), 2 (red) and 3 (green)
                    write_gpio(i,1) #turn on  LED
                    time.sleep(sleep_time) #sleep with ON
                    write_gpio(i,0) #turn off LED
                    if LED_Status[1] == 1: #if supposed to blink, sleep
                        time.sleep(sleep_time)
            else: #IF NO PWM & Solid Color
                write_gpio(LED_Status[2],1)
                time.sleep(sleep_time)
                if LED_Status[1] == 1: #if supposed to blink, sleep
                        write_gpio(LED_Status[2],0)
                        time.sleep(sleep_time)
        else: #If NOT full brightness (PWM needed)
            if LED_Status[2] == 0: #PWM & cycle thru colors
                for i in range(1,4):
                    for j in range(PWM_run_time):
                        write_gpio(i,1) #turn on LED
                        time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                        write_gpio(i,0) #turn off LED
                        time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                    if LED_Status[1] == 1: #If blinking
                        time.sleep(sleep_time)
            else:
                for i in range(PWM_run_time):
                    write_gpio(LED_Status[2],1) #turn on LED
                    time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                    write_gpio(LED_Status[2],0) #turn off LED
                    time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                if LED_Status[1] == 1: #If Blinking
                    time.sleep(sleep_time)

def LOOP(_l, num):
    global running
    while running:
        resource_available = _l.acquire(False)
        if resource_available == True: #Note: We always want the same "worker" to have the lock or the LED may not work properly
            LED()
            _l.release()
            #time.sleep(sleep_time-.1)
            #user_input(1)
        else:
            #INSERT FUNCTION TO SEE IF NEW MESSAGE CAME IN
            #This Function should set user_input = True
            print('Worker {} is searching for incoming message.'.format(num))
            time.sleep(sleep_time)

def Start_RGB():
    global threads
    reset_gpio()            

    fork = threading.Lock()
    for i in range(2):
        #fork is the shared lock
        #i is the user (led number too)
        t = threading.Thread(target=LOOP, args=(fork, i))
        threads.append(t) #creates the new thread number
        t.start()
        
def Stop_RGB():
    global running, threads
    running = False
    for t in threads:
        t.join()

In [4]:
def run_pynq_server_R():
    idx = 0
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('192.168.2.99', 12345))
    sock.listen()
    conn, addr = sock.accept()
    with conn:
        print("before pynq R")
        Start_RGB()
        while True: # TODO: update for different LED statuses
            time.sleep(0.0001)
            data = conn.recv(1)
            if (int(data.decode()) == 4):
                print("disconnect pynq R")
                Stop_RGB()
                break
            Change_LED(int(data.decode()), idx)
            idx += 1
            idx %= 4
    print("after pynq R")

run_pynq_server_R()

before pynq R
[23002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230, 0, 2, 3]
[23002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230, 2300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002300230023002

Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-4ba74a8801a3>", line 18, in run_pynq_server_R
    LED()
  File "<ipython-input-3-6fce0e5b6020>", line 37, in LED
    write_gpio(LED_Status[2],1)
  File "/usr/local/lib/python3.6/dist-packages/pynq/lib/pynqmicroblaze/magic.py", line 54, in __call__
    return self._function(*args)
  File "/usr/local/lib/python3.6/dist-packages/pynq/lib/pynqmicroblaze/rpc.py", line 747, in __call__
    self._call_function(*args)
  File "/usr/local/lib/python3.6/dist-packages/pynq/lib/pynqmicroblaze/rpc.py", line 736, in _call_function
    arg_string += self.function.pack_args(*args)
  File "/usr/local/lib/python3.6/dist-packages/pynq/lib/pynqmicroblaze/rpc.py", line 455, in pack_args
    self.arg_interfaces, args
  Fi